In [10]:
from DataBase.clean_database import database
from data_analisys.data_functions import *
import pandas as pd
import numpy as np

In [254]:
class Analisis:
    def __init__(self):
        self.df = database()
        self.loteria = self.df.copy()
        self.loteria['year'] = self.loteria.Dates.apply(lambda x: x.year)
        self.loteria = self.loteria.drop(columns=['Dates','Sorteo','Star_1','Star_2'])
        self.new_loteria = pd.DataFrame(columns=[i for i in range(1,51)],
                                        index=np.arange(self.loteria['year'].iloc[0],self.loteria['year'].iloc[-1]+1))
    
    def __transformacion_en_columnas(self,row):
        for sorteo in range(1,6):
            if not np.isnan(self.new_loteria.loc[row.year,row[f'Nro{sorteo}']]):
                self.new_loteria.loc[row.year,row[f'Nro{sorteo}']] += 1
            else:
                self.new_loteria.loc[row.year,row[f'Nro{sorteo}']] = 1
    
    def apply_transformacion(self):
        self.loteria.apply(self.__transformacion_en_columnas,axis=1)
        self.new_loteria.fillna(0,inplace=True)

        self.hits = self.new_loteria.sum().to_frame().rename(columns={0: 'hits'}).T.astype('int32')
        self.mean = self.new_loteria.mean().to_frame().rename(columns={0: 'mean'}).T.astype('int32')
        self.median = self.new_loteria.median().to_frame().rename(columns={0: 'median'}).T.astype('int32')

    def calculate_average_hits(self,is_star=False):
        divide = 5 if not is_star else 2
        average = round(self.hits.iloc[0][self.new_loteria.columns] / len(self.loteria) / divide, 6)
        self.average_hits = pd.DataFrame(columns = self.new_loteria.columns, 
                            data = [average], 
                            index = ['numbers/stars'])

In [255]:
euromillones = Analisis()
euromillones.apply_transformacion()
euromillones.calculate_average_hits()
